In [1]:
# SPDX-FileCopyrightText: Copyright (c) 2022 Guillaume Bellegarda. All rights reserved.
# SPDX-License-Identifier: BSD-3-Clause
# 
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
# 1. Redistributions of source code must retain the above copyright notice, this
# list of conditions and the following disclaimer.
#
# 2. Redistributions in binary form must reproduce the above copyright notice,
# this list of conditions and the following disclaimer in the documentation
# and/or other materials provided with the distribution.
#
# 3. Neither the name of the copyright holder nor the names of its
# contributors may be used to endorse or promote products derived from
# this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
# FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
# DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
# SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
# CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
# OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
#
# Copyright (c) 2022 EPFL, Guillaume Bellegarda

"""
Run stable baselines 3 on quadruped env 
Check the documentation! https://stable-baselines3.readthedocs.io/en/master/
"""
import os
from datetime import datetime
# stable baselines 3
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3 import PPO, SAC
from stable_baselines3.common.env_util import make_vec_env
# utils
from utils.utils import CheckpointCallback
from utils.file_utils import get_latest_model
# gym environment
from env.quadruped_gym_env import QuadrupedGymEnv


LEARNING_ALG = "PPO" # or "SAC"
LOAD_NN = False # if you want to initialize training with a previous model 
NUM_ENVS = 1    # how many pybullet environments to create for data collection
USE_GPU = False # make sure to install all necessary drivers 

LEARNING_ALG = "SAC";  USE_GPU = True
# after implementing, you will want to test how well the agent learns with your MDP: 
# env_configs = {"motor_control_mode":"CPG",
#                "task_env": "FWD_LOCOMOTION", #  "LR_COURSE_TASK",
#                "observation_space_mode": "LR_COURSE_OBS"}
env_configs = {}

if USE_GPU and LEARNING_ALG=="SAC":
    gpu_arg = "auto" 
else:
    gpu_arg = "cpu"

if LOAD_NN:
    interm_dir = "./logs/intermediate_models/"
    log_dir = interm_dir + '' # add path
    stats_path = os.path.join(log_dir, "vec_normalize.pkl")
    model_name = get_latest_model(log_dir)

# directory to save policies and normalization parameters
SAVE_PATH = './logs/intermediate_models/'+ datetime.now().strftime("%m%d%y%H%M%S") + '/'
os.makedirs(SAVE_PATH, exist_ok=True)
# checkpoint to save policy network periodically
checkpoint_callback = CheckpointCallback(save_freq=30000, save_path=SAVE_PATH,name_prefix='rl_model', verbose=2)
# create Vectorized gym environment
env = lambda: QuadrupedGymEnv(**env_configs)  
env = make_vec_env(env, monitor_dir=SAVE_PATH,n_envs=NUM_ENVS)
# normalize observations to stabilize learning (why?)
env = VecNormalize(env, norm_obs=True, norm_reward=False, clip_obs=100.)

if LOAD_NN:
    env = lambda: QuadrupedGymEnv(**env_configs)
    env = make_vec_env(env, monitor_dir=SAVE_PATH, n_envs=NUM_ENVS)
    env = VecNormalize.load(stats_path, env)

# Multi-layer perceptron (MLP) policy of two layers of size _,_ 
policy_kwargs = dict(net_arch=[256,256])
# What are these hyperparameters? Check here: https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html
n_steps = 4096 
learning_rate = lambda f: 1e-4 
ppo_config = {  "gamma":0.99, 
                "n_steps": int(n_steps/NUM_ENVS), 
                "ent_coef":0.0, 
                "learning_rate":learning_rate, 
                "vf_coef":0.5,
                "max_grad_norm":0.5, 
                "gae_lambda":0.95, 
                "batch_size":128,
                "n_epochs":10, 
                "clip_range":0.2, 
                "clip_range_vf":1,
                "verbose":1, 
                "tensorboard_log":None, 
                "_init_setup_model":True, 
                "policy_kwargs":policy_kwargs,
                "device": gpu_arg}

# What are these hyperparameters? Check here: https://stable-baselines3.readthedocs.io/en/master/modules/sac.html
sac_config={"learning_rate":1e-4,
            "buffer_size":300000,
            "batch_size":256,
            "ent_coef":'auto', 
            "gamma":0.99, 
            "tau":0.005,
            "train_freq":1, 
            "gradient_steps":1,
            "learning_starts": 10000,
            "verbose":1, 
            "tensorboard_log":None,
            "policy_kwargs": policy_kwargs,
            "seed":None, 
            "device": gpu_arg}

if LEARNING_ALG == "PPO":
    model = PPO('MlpPolicy', env, **ppo_config)
elif LEARNING_ALG == "SAC":
    model = SAC('MlpPolicy', env, **sac_config)
else:
    raise ValueError(LEARNING_ALG + 'not implemented')

if LOAD_NN:
    if LEARNING_ALG == "PPO":
        model = PPO.load(model_name, env)
    elif LEARNING_ALG == "SAC":
        model = SAC.load(model_name, env)
    print("\nLoaded model", model_name, "\n")

# Learn and save (may need to train for longer)
model.learn(total_timesteps=1000000, log_interval=1,callback=checkpoint_callback)
# Don't forget to save the VecNormalize statistics when saving the agent
model.save( os.path.join(SAVE_PATH, "rl_model" ) ) 
env.save(os.path.join(SAVE_PATH, "vec_normalize.pkl" )) 
if LEARNING_ALG == "SAC": # save replay buffer 
    model.save_replay_buffer(os.path.join(SAVE_PATH,"off_policy_replay_buffer"))



C:\Users\makss\anaconda3\envs\lr24\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


Using cpu device
=================================== Save path is ./logs/intermediate_models/120324141512/
=================================== Save path is ./logs/intermediate_models/120324141512/
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | 0.119    |
| time/              |          |
|    episodes        | 1        |
|    fps             | 165      |
|    time_elapsed    | 6        |
|    total_timesteps | 1001     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 560      |
|    ep_rew_mean     | 0.314    |
| time/              |          |
|    episodes        | 2        |
|    fps             | 154      |
|    time_elapsed    | 7        |
|    total_timesteps | 1120     |
---------------------------------
=================================== Save path is ./logs/intermediate_models/120324141512/
---------------------------------

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 702      |
|    ep_rew_mean     | 0.445    |
| time/              |          |
|    episodes        | 17       |
|    fps             | 90       |
|    time_elapsed    | 131      |
|    total_timesteps | 11932    |
| train/             |          |
|    actor_loss      | -68      |
|    critic_loss     | 7.36     |
|    ent_coef        | 0.826    |
|    ent_coef_loss   | -3.59    |
|    learning_rate   | 0.0001   |
|    n_updates       | 1931     |
---------------------------------
=================================== Save path is ./logs/intermediate_models/120324141512/
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 694      |
|    ep_rew_mean     | 0.455    |
| time/              |          |
|    episodes        | 18       |
|    fps             | 82       |
|    time_elapsed    | 150      |
|    total_timesteps | 12493    |
| train/             |    

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 499      |
|    ep_rew_mean     | 0.475    |
| time/              |          |
|    episodes        | 31       |
|    fps             | 58       |
|    time_elapsed    | 262      |
|    total_timesteps | 15458    |
| train/             |          |
|    actor_loss      | -159     |
|    critic_loss     | 12.5     |
|    ent_coef        | 0.59     |
|    ent_coef_loss   | -8.68    |
|    learning_rate   | 0.0001   |
|    n_updates       | 5457     |
---------------------------------
=================================== Save path is ./logs/intermediate_models/120324141512/
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 496      |
|    ep_rew_mean     | 0.477    |
| time/              |          |
|    episodes        | 32       |
|    fps             | 56       |
|    time_elapsed    | 278      |
|    total_timesteps | 15880    |
| train/             |    

=================================== Save path is ./logs/intermediate_models/120324141512/
=================================== Save path is ./logs/intermediate_models/120324141512/
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 506      |
|    ep_rew_mean     | 0.427    |
| time/              |          |
|    episodes        | 44       |
|    fps             | 42       |
|    time_elapsed    | 521      |
|    total_timesteps | 22251    |
| train/             |          |
|    actor_loss      | -208     |
|    critic_loss     | 86.6     |
|    ent_coef        | 0.303    |
|    ent_coef_loss   | -17.1    |
|    learning_rate   | 0.0001   |
|    n_updates       | 12250    |
---------------------------------
=================================== Save path is ./logs/intermediate_models/120324141512/
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 511      |
|    ep_rew_mean     | 0.423    |
| time/         

...

=================================== Save path is ./logs/intermediate_models/120324141512/
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 121      |
|    ep_rew_mean     | 10.3     |
| time/              |          |
|    episodes        | 1212     |
|    fps             | 24       |
|    time_elapsed    | 12028    |
|    total_timesteps | 299014   |
| train/             |          |
|    actor_loss      | -3.55    |
|    critic_loss     | 0.0213   |
|    ent_coef        | 0.000664 |
|    ent_coef_loss   | 3.55     |
|    learning_rate   | 0.0001   |
|    n_updates       | 289013   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 121      |
|    ep_rew_mean     | 10.3     |
| time/              |          |
|    episodes        | 1213     |
|    fps             | 24       |
|    time_elapsed    | 12034    |
|    total_timesteps | 299126   |
| train/             |    

=================================== Save path is ./logs/intermediate_models/120324141512/
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 131      |
|    ep_rew_mean     | 10.3     |
| time/              |          |
|    episodes        | 1226     |
|    fps             | 24       |
|    time_elapsed    | 12132    |
|    total_timesteps | 301508   |
| train/             |          |
|    actor_loss      | -3.59    |
|    critic_loss     | 0.0235   |
|    ent_coef        | 0.000663 |
|    ent_coef_loss   | 1.87     |
|    learning_rate   | 0.0001   |
|    n_updates       | 291507   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 131      |
|    ep_rew_mean     | 10.3     |
| time/              |          |
|    episodes        | 1227     |
|    fps             | 24       |
|    time_elapsed    | 12136    |
|    total_timesteps | 301605   |
| train/             |    

=================================== Save path is ./logs/intermediate_models/120324141512/
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 142      |
|    ep_rew_mean     | 10.4     |
| time/              |          |
|    episodes        | 1240     |
|    fps             | 24       |
|    time_elapsed    | 12238    |
|    total_timesteps | 304104   |
| train/             |          |
|    actor_loss      | -3.46    |
|    critic_loss     | 0.0156   |
|    ent_coef        | 0.000676 |
|    ent_coef_loss   | -2.68    |
|    learning_rate   | 0.0001   |
|    n_updates       | 294103   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 142      |
|    ep_rew_mean     | 10.4     |
| time/              |          |
|    episodes        | 1241     |
|    fps             | 24       |
|    time_elapsed    | 12244    |
|    total_timesteps | 304228   |
| train/             |    

=================================== Save path is ./logs/intermediate_models/120324141512/
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 143      |
|    ep_rew_mean     | 10.5     |
| time/              |          |
|    episodes        | 1254     |
|    fps             | 24       |
|    time_elapsed    | 12339    |
|    total_timesteps | 306571   |
| train/             |          |
|    actor_loss      | -3.41    |
|    critic_loss     | 0.0306   |
|    ent_coef        | 0.000658 |
|    ent_coef_loss   | -2.47    |
|    learning_rate   | 0.0001   |
|    n_updates       | 296570   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 143      |
|    ep_rew_mean     | 10.5     |
| time/              |          |
|    episodes        | 1255     |
|    fps             | 24       |
|    time_elapsed    | 12343    |
|    total_timesteps | 306684   |
| train/             |    

KeyboardInterrupt: 